In [ ]:
import random
import PIL
import bqplot
import io
import ipywidgets
import keras
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
x_train = train_images.reshape((60000, 28 * 28)).astype('float32') / 255
x_test = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

y_train = keras.utils.to_categorical(train_labels)
y_test = keras.utils.to_categorical(test_labels)

In [ ]:
r, c, s = 6, 8, 96
digits = train_images[np.random.randint(train_images.shape[0], size=r*c), :, :]
canvas = PIL.Image.new('RGB', (c*s+2, r*s+2), color='white')
for i,d in enumerate(digits):
    dimg = PIL.Image.fromarray(255-d).resize((s-8, s-8))
    canvas.paste(dimg, box=(s*int(i/r), s*(i%r)))

buf = io.BytesIO()
canvas.save(buf, 'gif')
img = ipywidgets.Image(value=buf.getvalue())
display(img)

In [ ]:
class plot_history(keras.callbacks.Callback):
    def __init__(self, loss_plt, acc_plt, mode='val', history = None):
        self.loss_plt = loss_plt
        self.acc_plt = acc_plt
        self.mode = mode
        if history is None:
            history = {'loss':[], 'val_loss':[], 'acc': [], 'val_acc':[]}
        self.history = history
    def on_epoch_end(self, epoch, logs={}):
        if self.mode == 'val':
            self.history['loss'].append(logs.get('loss'))
            self.history['val_loss'].append(logs.get('val_loss'))
            self.history['acc'].append(logs.get('acc'))
            self.history['val_acc'].append(logs.get('val_acc'))
        
        imbuf = BytesIO()

        plt.plot(range(1, len(self.history['loss'])+1), self.history['loss'], linewidth=2)
        plt.plot(range(1, len(self.history['val_loss'])+1), self.history['val_loss'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_loss')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Loss')
        plt.xlabel('Epochs')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.loss_plt.value = imbuf.getvalue()
        plt.clf()
        plt.plot(range(1, len(self.history['acc'])+1), self.history['acc'], linewidth=2)
        plt.plot(range(1, len(self.history['val_acc'])+1), self.history['val_acc'], linewidth=2)
        if self.mode == 'final':
            plt.plot([epoch+1], [logs.get('val_acc')], 'rx', ms=5.0)
            plt.axvline(epoch+1, ls=':')
        plt.ylabel('Accuracy')
        plt.xlabel('Steps')
        plt.savefig(imbuf)
        imbuf.seek(0)
        self.acc_plt.value = imbuf.getvalue()
        plt.clf()
        
        vt = "Validation" if self.mode == 'val' else "Test"
        print("Epoch {: 4d} | Loss: {:5.4f} Accuracy: {:5.4f} | {} Loss: {:5.4f} {} Accuracy {:5.4f}".format(
            epoch+1,logs.get('loss'),logs.get('acc'),vt,logs.get('val_loss'),vt,logs.get('val_acc')))

In [ ]:
def build_network():
    network = keras.models.Sequential(name='MNIST_DNN')
    network.add(keras.layers.Dense(32, activation='relu', name='input', input_shape=(28*28*1,)))
    network.add(keras.layers.Dense(32, activation='relu', name='hidden'))
    network.add(keras.layers.Dense(10, activation='softmax', name='output'))

    network.compile(optimizer=keras.optimizers.sgd(lr=0.01, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

    return network

In [ ]:
network = build_network()
network.summary()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

epochs = 50
history = network.fit(train_images, train_labels, epochs=epochs, 
                      batch_size=128, validation_split=0.25, verbose=0, callbacks=[plot_history(loss_plt, acc_plt)])

In [ ]:
network = build_network()
network.summary()

loss_plt = ipywidgets.Image()
acc_plt = ipywidgets.Image()
display(ipywidgets.HBox([loss_plt, acc_plt]))

epochs = 25
network.fit(train_images, train_labels, epochs=epochs, 
            batch_size=128, validation_data=((test_images, test_labels)), verbose=0, 
            callbacks=[plot_history(loss_plt, acc_plt, mode='final', history=history.history)])

In [ ]:
entry=9

prediction = network.predict(test_images[entry:entry+1])
predicted_label = np.argmax(prediction, axis=1)

img = test_images[entry].reshape((28,28))
plt.imshow(img, cmap=plt.cm.binary)
plt.show()
print("Ground truth: {} | Predicted: {}".format(np.argmax(test_labels[entry]),predicted_label[0]))
results = [{'label': l, 'score': s*100} for l,s in enumerate(prediction[0])]
print('-' * 20)
print("{0:>5} | {1:>6} ".format("Label", "Score"))
print('-' * 20)
for r in results:
    print("{label:>5} | {score:>6.2f}% ".format(**r))
print('-' * 20)